In [ ]:
import random
import numpy

def set_seed(seed: int, n_gpu: int):
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

In [ ]:
reviews_train = []
for line in open("../data/imdb/full_train.txt", "r"):
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open("../data/imdb/full_test.txt", "r"):
    reviews_test.append(line.strip())

In [ ]:
train_target = [1 if i < 12500 else 0 for i in range(25000)]
test_target = [1 if i < 12500 else 0 for i in range(25000)]

In [ ]:
from gensim.models import KeyedVectors
w2v_model = KeyedVectors.load_word2vec_format("../models/GoogleNews-vectors-negative300.bin", binary=True, limit=300000)

In [ ]:
UNKNOWN_TOKEN = "UNK"
PAD_TOKEN = "PAD"

index_to_token = [PAD_TOKEN, UNKNOWN_TOKEN] + w2v_model.index2word
token_to_index = {token: index + 2 for index, token in enumerate(w2v_model.index2word)}
token_to_index[PAD_TOKEN] = 0
token_to_index[UNKNOWN_TOKEN] = 1

In [ ]:
import torch
token_weights = torch.zeros((len(index_to_token), w2v_model.vector_size))
token_weights[2:, :] = torch.tensor(w2v_model.vectors)

In [ ]:
BATCH_SIZE = 20
MAX_INPUT_LENGTH = 500

In [ ]:
def generate_batch(input_data, max_length = MAX_INPUT_LENGTH):
    
    texts = torch.tensor([padding(sample["text"], max_length) for sample in input_data], dtype=torch.long)
    labels = torch.tensor([sample["label"] for sample in input_data], dtype=torch.long)
    return texts, torch.tensor([]), labels

def padding(text_tokens, max_length, padding_token=1):
    if len(text_tokens) >= max_length:
        return text_tokens[:max_length]
    return text_tokens + [padding_token]*(max_length - len(text_tokens))

In [ ]:
from nltk import word_tokenize

In [ ]:
prepared_data = []

for label, text in zip(train_target, reviews_train):
    text_tokens = [token_to_index.get(token.lower(), token_to_index[UNKNOWN_TOKEN]) for token in word_tokenize(text)]
    prepared_data.append({"label": label, "text": text_tokens, "sequence_length": len(text_tokens)})

In [ ]:
from torch.utils.data import DataLoader

def train(input_data, collate_fn, shuffle=True):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(input_data, batch_size=BATCH_SIZE, shuffle=shuffle, collate_fn=collate_fn)
    for i, (text, sequence_length, label) in enumerate(data): # 3
        optimizer.zero_grad()
        text, sequence_length, label = text.to(device), sequence_length.to(device), label.to(device)
        output = model(text, sequence_length)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == label).sum().item()

    return train_loss / len(input_data), train_acc / len(input_data)

def test(input_data, collate_fn):
    test_loss = 0
    acc = 0
    data = DataLoader(input_data, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    for i, (text, sequence_length, label) in enumerate(data):
        text, sequence_length, label = text.to(device),  sequence_length.to(device), label.to(device)
        with torch.no_grad():
            output = model(text, sequence_length)
            loss = criterion(output, label)
            test_loss += loss.item()
            acc += (output.argmax(1) == label).sum().item()

    return test_loss / len(input_data), acc / len(input_data)

## Simple RNN

In [ ]:
RNN_HIDDEN_SIZE = 100
IS_BIDIRECTIONAL = False
NUM_LAYERS = 1

In [ ]:
import torch
import torch.nn as nn

class SentimentClassificationSimpleRNNModel(nn.Module):
    def __init__(self, embedding_weights, num_class, hidden_size, is_bidirectional, num_layers):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_weights)
        self.embedding.weight.requires_grad = False
        embedding_dim = embedding_weights.size()[1]
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, bidirectional=is_bidirectional)
        self.fc = nn.Linear(hidden_size*(1 + int(is_bidirectional)), num_class)
        self.init_weights()

    def init_weights(self):
        init_range = 0.5
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()

    def forward(self, text, sequence_length=None):
        embedded = self.embedding(text)
        output, _ = self.rnn(embedded)
        pooled_output, _ = output.max(dim=1)
        return self.fc(pooled_output)

In [ ]:
set_seed(42, 1)

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = SentimentClassificationSimpleRNNModel(
    embedding_weights=token_weights, 
    num_class=len(set(train_target)), 
    hidden_size=RNN_HIDDEN_SIZE,
    is_bidirectional=IS_BIDIRECTIONAL,
    num_layers=NUM_LAYERS
).to(device)

In [ ]:
import time
from torch.utils.data.dataset import random_split

N_EPOCHS = 2
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

train_len = int(len(prepared_data) * 0.95)
train_data, validation_data = \
    random_split(prepared_data, [train_len, len(prepared_data) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train(train_data, generate_batch)
    valid_loss, valid_acc = test(validation_data, generate_batch)

    secs = int(time.time() - start_time)

    print(f"Epoch: {(epoch + 1)} | time in {secs} seconds")
    print(f"Loss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)")
    print(f"Loss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)")

## Masked Padding

У нас есть два предложения: "I like watching movies" and "I don't like watching movies". После токенизации мы получим: `["I", "like", "watching", "movies"]` (4 токена) и `["I", "don't", "like", "watching", "movies"]` (5 токенов).

Чтобы сформировать батч, оба предложения должны иметь одинаковое количество токенов, поэтому мы добавляем к первому предложению «padding токен» и получаем
`["I", "like", "watching", "movies", "PAD"]` (5 токенов) и `["I", "don't", "like", "watching", "movies"]` (5 токенов)

Теперь наша модель будет знать, что мы использовали padding для первого предложения.
Должна ли наша модель знать об этом?

Мы можем маскировать токены, и наша модель их проигнорирует. Использовать `torch.nn.utils.rnn.pack_padded_sequence`

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class SentimentClassificationMaskedRNNModel(nn.Module):
    def __init__(self, embedding_weights, num_class, hidden_size, is_bidirectional, num_layers):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_weights)
        self.embedding.weight.requires_grad = False
        embedding_dim = embedding_weights.size()[1]
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, bidirectional=is_bidirectional)
        self.fc = nn.Linear(hidden_size*(1 + int(is_bidirectional)), num_class)
        self.init_weights()

    def init_weights(self):
        init_range = 0.5
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()
        
    def forward(self, text, sequence_length):
        embedded = self.embedding(text)
        packed_output, _ = self.rnn(pack_padded_sequence(embedded, sequence_length, batch_first=True))
        output, _ = pad_packed_sequence(packed_output)
        pooled_output, _ = output.max(dim=0)
        return self.fc(pooled_output)

In [ ]:
from typing import Dict

def padding(text_tokens, max_length, padding_token=token_to_index[PAD_TOKEN]):
    if len(text_tokens) >= max_length:
        return text_tokens[:max_length]
    return text_tokens + [padding_token]*(max_length - len(text_tokens))

def generate_batch_with_masking(input_data: Dict[str, str], max_length = MAX_INPUT_LENGTH):
    # return (text, sequence_length), labels
    max_batch_sequence_length = max(sequence["sequence_length"] for sequence in input_data)
    batch_sequence_length = min(max_length, max_batch_sequence_length)
    prepared_data = []
    sequence_length = []
    for sample in input_data:
        prepared_data.append(padding(sample["text"], batch_sequence_length))
        sequence_length.append(min(sample["sequence_length"], batch_sequence_length))
        
    prepared_data = torch.tensor(prepared_data, dtype=torch.long)
    sequence_length = torch.tensor(sequence_length, dtype=torch.long)
    sequence_length, perm_idx = sequence_length.sort(0, descending=True)
    prepared_data = prepared_data[perm_idx]
    
    labels = torch.tensor([sample["label"] for sample in input_data], dtype=torch.long)[perm_idx]
    return prepared_data, sequence_length, labels

In [ ]:
generate_batch_with_masking(
    [
        {"text": [1, 2, 3, 4], "label": 1, "sequence_length": 4}, 
        {"text": [1, 5, 2, 3, 4], "label": 0, "sequence_length": 5}, 
    ]
)

In [ ]:
set_seed(42, 1)

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = SentimentClassificationMaskedRNNModel(
    embedding_weights=token_weights,
    num_class=len(set(train_target)), 
    hidden_size=RNN_HIDDEN_SIZE,
    is_bidirectional=IS_BIDIRECTIONAL,
    num_layers=NUM_LAYERS
).to(device)

In [ ]:
import time
from torch.utils.data.dataset import random_split

N_EPOCHS = 2
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

train_len = int(len(prepared_data) * 0.95)
train_data, validation_data = \
    random_split(prepared_data, [train_len, len(prepared_data) - train_len])
sorted_train_data = sorted(train_data, key=lambda x: x["sequence_length"], reverse=True)
sorted_val_data = sorted(validation_data, key=lambda x: x["sequence_length"], reverse=True)

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(sorted_train_data, generate_batch_with_masking, False)
    valid_loss, valid_acc = test(sorted_val_data, generate_batch_with_masking)

    secs = int(time.time() - start_time)

    print(f"Epoch: {(epoch + 1)} | time in {secs} seconds")
    print(f"Loss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)")
    print(f"Loss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)")